# <div align="center" style="color: #ff5733;">Data For Finance Team</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 4

In [3]:
sq = """SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2
"""
StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID ab3fe240-3694-4944-879f-6dd79a8175ee successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [4]:
sq = """
SELECT 
    DATE_TRUNC( termsAndConditionsSubmitDateTime,MONTH) as mm,  
    (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in SubmittedApps dataframe are:\t {SubmittedApps.shape}")

Job ID e35da770-2f3f-4e86-8635-133d5c238932 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in SubmittedApps dataframe are:	 (159, 3)


In [5]:
sq = """
SELECT 
    DATE_TRUNC( decision_date,MONTH) as mm, 
    (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where flagApproval = 1

group by 1,2
order by 1,2
"""
ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps dataframe are:\t {ApprovedApps.shape}")

Job ID d8b35065-3f2d-404f-ac87-1089b68f8532 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps dataframe are:	 (112, 3)


In [6]:
# BookedApps
sq = """SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t{BookedApps.shape}")

Job ID cc1fd222-f1ab-48ff-a8aa-a664dd7620e3 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	(149, 3)


In [7]:
# %%bigquery BookedAmt

sq = """
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1,2
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t{BookedAmt.shape}")

Job ID 9bf515be-b013-4566-8a02-1c6227d312d2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	(149, 3)


In [8]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [9]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

### Loan Account Tagging

In [10]:
# %%bigquery loan_type
sq = """
  SELECT
    loanAccountNumber,
    (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct,
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
""" 

loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type dataframe are:\t {loan_type.shape}")


Job ID a0ecb7f5-4a34-41cf-9411-a32fc95178ad successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type dataframe are:	 (126868, 4)


In [11]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [12]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct', 'OriginalLoanProduct']]

#### VAS

In [13]:
# %%bigquery vas_df

sq = """
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
     (CASE WHEN reloan_flag = 1 then "Reloan" else  new_loan_type END) AS new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2,3 
order by 1,2,3
"""
vas_df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

print(f"The rows and columns in vas_df dataframe are:\t {vas_df.shape}")

Job ID 70dc559c-03ac-4aab-ae53-6db738dbd40b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df dataframe are:	 (143, 5)


In [14]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [16]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)